In [2]:
import pandas as pd
import openpyxl
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
file_path = '../data/clean/Global_Super_Store.csv'
print(f"Using file path: '{file_path}'")

Using file path: '../data/clean/Global_Super_Store.csv'


Loading & reading

In [32]:
# Load the Excel file
data = pd.read_csv(file_path, encoding='latin1')

In [33]:
data.shape

(51290, 25)

Check empty(s)

In [34]:
data.head
data.isnull().sum() 

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
Product ID            0
Category              0
Sub-Category          0
Product Name          0
Price                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
Sales                 0
dtype: int64

For my purposes, missing postal codes OK

Start Cleaning

In [35]:
# Handle missing values
# Fill missing numeric values with the mean
data['Sales'].fillna(data['Sales'].mean(), inplace=True)

#Convert Date columns to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], format='%d-%m-%Y')

data.columns.str.strip()

C:\Users\fabia\AppData\Local\Temp\ipykernel_40800\559077648.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sales'].fillna(data['Sales'].mean(), inplace=True)


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Price', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority', 'Sales'],
      dtype='object')

In [36]:
data = data[data['Sales'] > 0]


In [37]:
#Ensure correct data types for numeric columns
data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce')
data['Profit'] = pd.to_numeric(data['Profit'], errors='coerce')
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

#Convert categorical columns to category data type
categorical_columns = ['Ship Mode', 'Customer ID', 'Segment', 'City', 'State', 'Category', 'Sub-Category', 'Product Name', 'Order Priority']
for col in categorical_columns:
    data[col] = data[col].astype('category')

#Check for consistency in categorical variables (e.g., capitalization in 'State')
data['State'] = data['State'].str.strip().str.title()

Check data shape after cleaning

In [38]:
data.shape

(51290, 25)

In [39]:
data.rename(columns={
    'Product ID': 'ProductID', 
    'Customer Name': 'FullName',
    'Order Priority': 'OrderPriority',
    'Order Date': 'OrderDate',
    'Ship Mode': 'ShippingMode',
    'Quantity': 'Quantity',
    'Segment': 'Segment',
    'Sales': 'Price',
    'Discount': 'Discount',
    'Profit': 'Profit',
    'Address ID': 'AddressID',
    'City': 'City',
    'State': 'State',
    'Country': 'Country',
    'Postal Code': 'PostalCode',
    'Region': 'Region',
    'Ship Date': 'ShippingDate',
    'Ship Cost': 'ShippingCost'
}, inplace=True)

In [40]:
display(data)

,Row ID,Order ID,OrderDate,ShippingDate,ShippingMode,Customer ID,FullName,Segment,City,State,...,Category,Sub-Category,Product Name,Price,Quantity,Discount,Profit,Shipping Cost,OrderPriority,Price
0,1,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,...,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,0.0,1.03,Medium,39.24
1,2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,0.0,13.45,Medium,2017.28
2,3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,0.0,9.63,Medium,386.56
3,4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,0.0,1.37,Medium,141.76
4,5,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,0.0,3.79,Medium,143.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Binders,"Avery Binder Covers, Economy",11.07,1,0.0,0.0,1.98,High,11.07
51286,51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Technology,Accessories,"Logitech Flash Drive, USB",61.44,2,0.0,0.0,13.02,High,122.88
51287,51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Art,"Boston Highlighters, Fluorescent",80.52,4,0.0,0.0,8.78,High,322.08
51288,51289,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.0,0.0,18.97,High,521.76


Realized that 'Sales' should be 'Price' column

In [41]:
#data.rename(columns={'Sales': 'Price'}, inplace=True)

In [42]:
display(data)

,Row ID,Order ID,OrderDate,ShippingDate,ShippingMode,Customer ID,FullName,Segment,City,State,...,Category,Sub-Category,Product Name,Price,Quantity,Discount,Profit,Shipping Cost,OrderPriority,Price
0,1,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,...,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,0.0,1.03,Medium,39.24
1,2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,0.0,13.45,Medium,2017.28
2,3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,0.0,9.63,Medium,386.56
3,4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,0.0,1.37,Medium,141.76
4,5,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,0.0,3.79,Medium,143.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Binders,"Avery Binder Covers, Economy",11.07,1,0.0,0.0,1.98,High,11.07
51286,51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Technology,Accessories,"Logitech Flash Drive, USB",61.44,2,0.0,0.0,13.02,High,122.88
51287,51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Art,"Boston Highlighters, Fluorescent",80.52,4,0.0,0.0,8.78,High,322.08
51288,51289,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,Office Supplies,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.0,0.0,18.97,High,521.76


Export to CSV

In [43]:

#data.to_csv('../data/clean/Global_Super_Store.csv', index=False)

In [44]:
# Check the first few rows of the DataFrame
print(data.head())

   Row ID        Order ID  OrderDate ShippingDate    ShippingMode Customer ID  \
0       1  MX-2014-143658 2014-10-02   2014-10-06  Standard Class    SC-20575   
1       2  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
2       3  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
3       4  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
4       5  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   

         FullName   Segment           City             State  ...  \
0    Sonia Cooley  Consumer    Mexico City  Distrito Federal  ...   
1  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
2  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
3  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
4  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   

          Category  Sub-Category                        Product Name   Price  \
0  Office Supplies

In [45]:
username = 'root'
password = 'q1w2e3r4'
host = '127.0.0.1'
database_name = 'globalsuperstore'

# Connection string format: 'mysql+pymysql://<username>:<password>@<host>/<database_name>'
connection_string = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

# Create an engine to connect to the MySQL database
engine = create_engine(connection_string)

Write dataframe to SQL

In [46]:
# Test
# data.to_sql('table_name', con=engine, if_exists='replace', index=False)

In [4]:
# Load CSV
data = pd.read_csv('../data/raw/Global_Super_Store.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/Global_Super_Store.csv'

In [ ]:
# Replace None or NaN values with empty strings in relevant columns
cleaned_data.fillna('', inplace=True)

# Save the cleaned file
cleaned_file_path = '/mnt/data/cleaned_customers.csv'
cleaned_data[['CustomerID', 'CustomerName', 'Segment', 'Region']].to_csv(cleaned_file_path, index=False)
cleaned_file_path

In [49]:
# Exploratory Data Analysis - Visualization
plt.figure(figsize=(12, 6))
sns.histplot(data['Sales'], bins=50, kde=True)
plt.title('Sales Distribution')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='Category', y='Profit', data=cleaned_data)
plt.title('Profit Distribution by Category')
plt.xlabel('Category')
plt.ylabel('Profit')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x='Ship Mode', data=cleaned_data)
plt.title('Count of Orders by Shipping Mode')
plt.xlabel('Shipping Mode')
plt.ylabel('Count')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x='Region', y='Sales', data=cleaned_data, estimator=sum)
plt.title('Total Sales by Region')
plt.xlabel('Region')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

KeyError: 'Sales'

<Figure size 1200x600 with 0 Axes>